In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('data.csv')

data.set_index('shot_id', inplace=True)
data["action_type"] = data["action_type"].astype('object')
data["combined_shot_type"] = data["combined_shot_type"].astype('category')
data["game_event_id"] = data["game_event_id"].astype('category')
data["game_id"] = data["game_id"].astype('category')
data["period"] = data["period"].astype('object')
data["playoffs"] = data["playoffs"].astype('category')
data["season"] = data["season"].astype('category')
data["shot_made_flag"] = data["shot_made_flag"].astype('category')
data["shot_type"] = data["shot_type"].astype('category')
data["team_id"] = data["team_id"].astype('category')

In [3]:
data.head(2)


,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_made_flag,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent
shot_id,,,,,,,,,,,,,,,,,,,,,
1,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,NaN,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR
2,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,0.0,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR


In [4]:
print data.shape
unknown_mask = data['shot_made_flag'].isnull()

(30697, 24)


# Data Cleaning


In [9]:
data_cl = data.copy() # create a copy of data frame
target = data_cl['shot_made_flag'].copy()

# Remove some columns
data_cl.drop('team_id', axis=1, inplace=True) # Always one number
data_cl.drop('lat', axis=1, inplace=True) # Correlated with loc_x
data_cl.drop('lon', axis=1, inplace=True) # Correlated with loc_y
data_cl.drop('game_id', axis=1, inplace=True) # Independent
data_cl.drop('game_event_id', axis=1, inplace=True) # Independent
data_cl.drop('team_name', axis=1, inplace=True) # Always LA Lakers
data_cl.drop('shot_made_flag', axis=1, inplace=True)

# Remaining time
data_cl['seconds_from_period_end'] = 60 * data_cl['minutes_remaining'] + data_cl['seconds_remaining']
data_cl['last_5_sec_in_period'] = data_cl['seconds_from_period_end'] < 5

data_cl.drop('minutes_remaining', axis=1, inplace=True)
data_cl.drop('seconds_remaining', axis=1, inplace=True)
data_cl.drop('seconds_from_period_end', axis=1, inplace=True)

## Matchup - (away/home)
data_cl['home_play'] = data_cl['matchup'].str.contains('vs').astype('int')
data_cl.drop('matchup', axis=1, inplace=True)

# Game date
data_cl['game_date'] = pd.to_datetime(data_cl['game_date'])
data_cl['game_year'] = data_cl['game_date'].dt.year
data_cl['game_month'] = data_cl['game_date'].dt.month
data_cl.drop('game_date', axis=1, inplace=True)


In [10]:
categorial_cols = [
    'action_type', 'combined_shot_type', 'period', 'season', 'shot_type',
    'shot_zone_area', 'shot_zone_basic', 'shot_zone_range', 'game_year',
    'game_month', 'opponent', 'loc_x', 'loc_y']

for cc in categorial_cols:
    dummies = pd.get_dummies(data_cl[cc])
    dummies = dummies.add_prefix("{}#".format(cc))
    data_cl.drop(cc, axis=1, inplace=True)
    data_cl = data_cl.join(dummies)

In [11]:
data_cl.head(10)

,playoffs,shot_distance,last_5_sec_in_period,home_play,action_type#Alley Oop Dunk Shot,action_type#Alley Oop Layup shot,action_type#Cutting Finger Roll Layup Shot,action_type#Cutting Layup Shot,action_type#Driving Bank shot,action_type#Driving Dunk Shot,...,loc_y#664,loc_y#679,loc_y#696,loc_y#697,loc_y#702,loc_y#711,loc_y#728,loc_y#741,loc_y#773,loc_y#791
shot_id,,,,,,,,,,,,,,,,,,,,,
1,0,18,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,15,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,16,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,22,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,False,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,0,14,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,2,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,12,False,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
